In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
import pandas as pd
import numpy as np

2021-08-24 11:29:44,051 : INFO : NumExpr defaulting to 4 threads.


In [3]:
df_full = pd.read_csv('./full.csv')
df_full.shape

(490, 7)

In [106]:
df_full.sample(10)

,Unnamed: 0,url,date,title,keywords,category,text
395,395,https://www.bloomberg.com/news/articles/2020-0...,2020-05-29T08:00:00.004Z,Airlines Leave $29 Billion Aid Fund Untapped i...,"['Saleha Mohsin', 'Ryan Beene']",economics,A pilot pulls luggage while walking through a ...
305,305,https://www.bloomberg.com/news/articles/2021-0...,2021-07-16T20:10:59.012Z,Virgin Galactic Shares Come Back to Earth Afte...,['Esha Dey'],Markets,Richard Branson’s rocket ride last weekend wen...
54,54,https://www.bloomberg.com/news/articles/2021-0...,2021-02-10T09:54:57.870Z,Etihad’s Focus on Smaller Aircraft Puts Fleet ...,"['Layan Odeh', 'Manus Cranny']",Business,Etihad Airways plans to rebuild operations aro...
298,298,https://www.bloomberg.com/news/articles/2021-0...,2021-02-01T23:33:37.077Z,Airline Bailout Begins in Canada With Loan to ...,"['Sandrine Rastello', 'Derek Decloet']",Business,Canada granted an emergency loan to a company ...
225,225,https://www.bloomberg.com/news/articles/2020-1...,2020-10-27T06:07:18.852Z,Japanese Airline ANA Forecasts Its Biggest Eve...,"['Shiho Takezawa', 'Tsuyoshi Inajima']",Business,ANA shares fell 3.2% before the results and re...
362,362,https://www.bloomberg.com/news/articles/2020-0...,2020-08-12T04:37:12.598Z,Cathay Rises Most Since 2008 as Reopening Chan...,['Will Davies'],Business,Cathay Pacific Airways Ltd. shares climbed the...
435,435,https://www.bloomberg.com/news/articles/2021-0...,2021-04-17T04:00:01.000Z,Vaccinated Countries Wary of Relapse Stunt Air...,['Siddharth Vikram Philip'],Markets,Countries that have quickly rolled out coronav...
461,461,https://www.bloomberg.com/news/articles/2021-0...,2021-02-23T22:54:24.411Z,Shot Hailed as Super Effective; J&J Gets Big W...,[],prognosis,The shot made by Pfizer Inc. and BioNTech SE w...
275,275,https://www.bloomberg.com/news/articles/2021-0...,2021-07-15T21:58:48.970Z,Biden Signals That Ban on Europe Travel Will B...,"['Jennifer Epstein', 'Christopher Jasper', 'Pa...",Politics,Joe Biden speaks during a news conference at t...
14,14,https://www.bloomberg.com/news/articles/2021-0...,2021-06-09T09:35:21.209Z,Ryanair Wins EU Court Challenge to Condor’s Co...,['Stephanie Bodoni'],Markets,Ryanair Holdings Plc notched up a third win in...


In [112]:
df_full.iloc[395].text

'A pilot pulls luggage while walking through a terminal at San Francisco International Airport, on April 2. U.S. airlines have yet to tap $29 billion in federal pandemic relief loans as they wait to see whether the reopening of the economy revives demand and diminishes the need for money that comes with government strings attached. Although the four largest U.S. passenger airlines have applied for the Treasury Department program, only American Airlines Group Inc. has said it intends to tap the pool of funds. Southwest Airlines Co., United Airlines Holdings and Delta Air Lines Inc. say they plan to wait until fall before deciding whether to take the money -- after a summer travel season that could see more people return to the skies.'

In [ ]:
# df_train = pd.read_csv('./train.csv')
# df_train.shape

In [ ]:
# df_test = pd.read_csv('./test.csv')
# df_test.shape

In [ ]:
# # dev
# df_train = df_train.sample(50, random_state=42)
# df_test = df_test.sample(50, random_state=42)

In [ ]:
# df_train.shape, df_test.shape

In [ ]:
# df_train.head(2)

In [ ]:
# corpus_text_train = df_train.text.tolist()

In [4]:
corpus_text_full = df_full.text.tolist()

In [ ]:
# print(corpus_text_train[-1])

In [7]:
print(corpus_text_full[-1][:200])

Few restrictions exist around tourism, but the megalopolis is still facing a brutal third wave of Covid-19 infections. The Palace of Fine Arts (Palacio de Bellas Artes) is a prominent cultural center 


In [ ]:
# def friendly_tag_corpus_train(row):
#     doc_categories = row.categories.split('\n')
#     doc_title = row.title
#     return [doc_title, *doc_categories]

In [64]:
import ast

def friendly_tag_corpus_full(row):
    if str(row.category) == "nan":
        doc_category = None
    else:
        doc_category = row.category
    if not doc_category:
        doc_category = None
    doc_title = row.title
    if not doc_title:
        doc_title = row.text[:100]
    doc_keywords = ast.literal_eval(row.keywords)
    return [
        f'title:{doc_title}',
        *[f'category:{d}' for d in [doc_category] if d is not None],
        *[f'keyword:{d}' for d in doc_keywords if d is not None]
    ]

In [65]:
# corpus_tags_friendly_train = df_train[["title", "categories"]].apply(friendly_tag_corpus_train, axis=1).to_list()

In [66]:
corpus_tags_friendly_full = df_full[["title", "category", "keywords"]].apply(friendly_tag_corpus_full, axis=1).to_list()

In [ ]:
# corpus_tags_friendly_train[-1]

In [72]:
corpus_tags_friendly_full[-6]

['title:Another Regional Airline Falls to the Covid-19 Recession',
 'category:Business',
 'keyword:Justin Bachman']

In [ ]:
# len(corpus_tags_friendly_train), len(corpus_text_train)

In [73]:
len(corpus_tags_friendly_full), len(corpus_text_full)

(490, 490)

In [74]:
def build_tag_id_mapping(corpus_tags):
    tags = list(set(tag for tags in corpus_tags for tag in tags))
    return {tag: idx for idx, tag in enumerate(tags)}

In [ ]:
# tag_id_mapping = build_tag_id_mapping(corpus_tags_friendly_train)
# id_tag_mapping = {v: k for k, v in tag_id_mapping.items()}

In [75]:
tag_id_mapping = build_tag_id_mapping(corpus_tags_friendly_full)
id_tag_mapping = {v: k for k, v in tag_id_mapping.items()}

In [ ]:
# corpus_tags_train = [[tag_id_mapping[tag] for tag in tags] for tags in corpus_tags_friendly_train]

In [76]:
corpus_tags_full = [[tag_id_mapping[tag] for tag in tags] for tags in corpus_tags_friendly_full]

In [ ]:
# len(corpus_tags_train)

In [77]:
len(corpus_tags_full)

490

In [78]:
from tqdm.notebook import tqdm_notebook

In [79]:
from gensim.utils import simple_preprocess

def corpus2tokens(raw_corpus):
    return [simple_preprocess(doc) for doc in tqdm_notebook(raw_corpus)]

In [ ]:
# %%time
# corpus_words_train = corpus2tokens(corpus_text_train)

In [80]:
%%time
corpus_words_full = corpus2tokens(corpus_text_full)


CPU times: user 427 ms, sys: 18.6 ms, total: 446 ms
Wall time: 435 ms


In [ ]:
# corpus_words_train[-1][:20]

In [81]:
corpus_words_full[-1][:20]

['few',
 'restrictions',
 'exist',
 'around',
 'tourism',
 'but',
 'the',
 'megalopolis',
 'is',
 'still',
 'facing',
 'brutal',
 'third',
 'wave',
 'of',
 'covid',
 'infections',
 'the',
 'palace',
 'of']

In [ ]:
# corpus_tags_train[-1]

In [82]:
corpus_tags_full[-1]

[602, 420, 499]

In [ ]:
# [id_tag_mapping[tag] for tag in corpus_tags_train[-1]]

In [83]:
[id_tag_mapping[tag] for tag in corpus_tags_full[-1]]

['title:What It’s Like to Visit Mexico City Now',
 'category:Pursuits',
 'keyword:Cyntia Aurora Barrera Diaz']

In [84]:
len(id_tag_mapping)

757

In [85]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [86]:
def gen_tagged_docs(corpus_words, corpus_tags):
    return [TaggedDocument(doc_words, doc_tags) for doc_words, doc_tags in zip(corpus_words, corpus_tags)]

In [ ]:
# corpus_train = gen_tagged_docs(corpus_words_train, corpus_tags_train)

In [87]:
corpus_full = gen_tagged_docs(corpus_words_full, corpus_tags_full)

In [ ]:
# corpus_train[-1]

In [88]:
corpus_full[-1]

TaggedDocument(words=['few', 'restrictions', 'exist', 'around', 'tourism', 'but', 'the', 'megalopolis', 'is', 'still', 'facing', 'brutal', 'third', 'wave', 'of', 'covid', 'infections', 'the', 'palace', 'of', 'fine', 'arts', 'palacio', 'de', 'bellas', 'artes', 'is', 'prominent', 'cultural', 'center', 'in', 'mexico', 'city', 'hosting', 'many', 'exhibitions', 'and', 'theatrical', 'performances', 'cyntia', 'barrera', 'diaz', 'is', 'bloomberg', 'breaking', 'news', 'editor', 'based', 'in', 'mexico', 'city', 'on', 'most', 'weekends', 'in', 'the', 'trendy', 'condesa', 'roma', 'and', 'polanco', 'neighborhoods', 'of', 'mexico', 'city', 'restaurants', 'outdoor', 'seats', 'are', 'packed', 'with', 'long', 'lingering', 'patrons', 'and', 'their', 'pets', 'unfazed', 'by', 'the', 'traffic', 'passing', 'just', 'inches', 'away', 'joggers', 'can', 'be', 'seen', 'training', 'in', 'the', 'early', 'morning', 'winding', 'through', 'the', 'expansive', 'chapultepec', 'park', 'and', 'down', 'tree', 'lined', 'ref

In [94]:
# https://groups.google.com/g/gensim/c/6JmSsx4iIv0
# projects with larger vocabularies tend to lean more towards negative-sampling than hierarchical-softmax
# VERY NB - https://stackoverflow.com/a/37502976/1782641
# https://radimrehurek.com/gensim/models/doc2vec.html
model = Doc2Vec(
    vector_size=1000,
    epochs=200,
    min_count=10,
    window=10,
    hs=0,
    negative=20,
    sample=1e-3,
    workers=3  # 64
)

2021-08-24 12:17:22,988 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d1000,n20,w10,mc10,s0.001,t3)', 'datetime': '2021-08-24T12:17:22.988090', 'gensim': '4.0.1', 'python': '3.8.10 (default, Jun  4 2021, 15:09:15) \n[GCC 7.5.0]', 'platform': 'Linux-5.13.8-100.fc33.x86_64-x86_64-with-glibc2.17', 'event': 'created'}


In [ ]:
# %%time
# model.build_vocab(corpus_train)

In [95]:
%%time
model.build_vocab(corpus_full)

2021-08-24 12:17:26,681 : INFO : collecting all words and their counts
2021-08-24 12:17:26,682 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
--- Logging error ---
Traceback (most recent call last):
  File "/home/dnk8n/miniconda3/envs/internship/lib/python3.8/logging/__init__.py", line 1085, in emit
    msg = self.format(record)
  File "/home/dnk8n/miniconda3/envs/internship/lib/python3.8/logging/__init__.py", line 929, in format
    return fmt.format(record)
  File "/home/dnk8n/miniconda3/envs/internship/lib/python3.8/logging/__init__.py", line 668, in format
    record.message = record.getMessage()
  File "/home/dnk8n/miniconda3/envs/internship/lib/python3.8/logging/__init__.py", line 373, in getMessage
    msg = msg % self.args
TypeError: not enough arguments for format string
Call stack:
  File "/home/dnk8n/miniconda3/envs/internship/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/hom

CPU times: user 178 ms, sys: 40.8 ms, total: 218 ms
Wall time: 199 ms


In [ ]:
# print(f"Word 'airport' appeared {model.wv.get_vecattr('airport', 'count')} times in the training corpus.")

In [96]:
print(f"Word 'airport' appeared {model.wv.get_vecattr('airport', 'count')} times in the full corpus.")

Word 'airport' appeared 201 times in the full corpus.


In [ ]:
# %%time
# model.train(corpus_train, total_examples=model.corpus_count, epochs=model.epochs)

In [97]:
%%time
model.train(corpus_full, total_examples=model.corpus_count, epochs=model.epochs)

2021-08-24 12:17:31,735 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 2898 vocabulary and 1000 features, using sg=0 hs=0 sample=0.001 negative=20 window=10', 'datetime': '2021-08-24T12:17:31.734979', 'gensim': '4.0.1', 'python': '3.8.10 (default, Jun  4 2021, 15:09:15) \n[GCC 7.5.0]', 'platform': 'Linux-5.13.8-100.fc33.x86_64-x86_64-with-glibc2.17', 'event': 'train'}
2021-08-24 12:17:32,782 : INFO : EPOCH 1 - PROGRESS: at 45.10% examples, 74656 words/s, in_qsize 5, out_qsize 0
2021-08-24 12:17:33,880 : INFO : EPOCH 1 - PROGRESS: at 80.00% examples, 66672 words/s, in_qsize 5, out_qsize 0
2021-08-24 12:17:34,158 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:17:34,217 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-08-24 12:17:34,234 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:17:34,235 : INFO : EPOCH - 1 : training on 260894 raw words (178577 effective words) took 

2021-08-24 12:18:05,648 : INFO : EPOCH 12 - PROGRESS: at 100.00% examples, 57892 words/s, in_qsize 0, out_qsize 1
2021-08-24 12:18:05,650 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:18:05,655 : INFO : EPOCH - 12 : training on 260894 raw words (178621 effective words) took 3.1s, 57764 effective words/s
2021-08-24 12:18:06,735 : INFO : EPOCH 13 - PROGRESS: at 29.39% examples, 43142 words/s, in_qsize 5, out_qsize 0
2021-08-24 12:18:07,888 : INFO : EPOCH 13 - PROGRESS: at 62.45% examples, 49427 words/s, in_qsize 5, out_qsize 0
2021-08-24 12:18:08,900 : INFO : EPOCH 13 - PROGRESS: at 93.06% examples, 50305 words/s, in_qsize 3, out_qsize 0
2021-08-24 12:18:09,059 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:18:09,112 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-08-24 12:18:09,163 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:18:09,164 : INFO : EPOCH - 13 : tr

2021-08-24 12:18:42,274 : INFO : EPOCH 24 - PROGRESS: at 66.12% examples, 57009 words/s, in_qsize 5, out_qsize 0
2021-08-24 12:18:43,240 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:18:43,290 : INFO : EPOCH 24 - PROGRESS: at 98.57% examples, 57202 words/s, in_qsize 1, out_qsize 1
2021-08-24 12:18:43,294 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-08-24 12:18:43,299 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:18:43,301 : INFO : EPOCH - 24 : training on 260894 raw words (178723 effective words) took 3.1s, 58205 effective words/s
2021-08-24 12:18:44,390 : INFO : EPOCH 25 - PROGRESS: at 28.37% examples, 42676 words/s, in_qsize 5, out_qsize 0
2021-08-24 12:18:45,399 : INFO : EPOCH 25 - PROGRESS: at 57.14% examples, 49609 words/s, in_qsize 5, out_qsize 0
2021-08-24 12:18:46,270 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:18:46,300 : INFO : worker thread fi

2021-08-24 12:19:14,964 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:19:14,965 : INFO : EPOCH - 36 : training on 260894 raw words (178588 effective words) took 2.1s, 84839 effective words/s
2021-08-24 12:19:15,983 : INFO : EPOCH 37 - PROGRESS: at 48.57% examples, 82697 words/s, in_qsize 5, out_qsize 0
2021-08-24 12:19:16,946 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:19:16,964 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-08-24 12:19:16,985 : INFO : EPOCH 37 - PROGRESS: at 100.00% examples, 88520 words/s, in_qsize 0, out_qsize 1
2021-08-24 12:19:16,986 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:19:16,986 : INFO : EPOCH - 37 : training on 260894 raw words (178597 effective words) took 2.0s, 88453 effective words/s
2021-08-24 12:19:18,023 : INFO : EPOCH 38 - PROGRESS: at 48.57% examples, 81248 words/s, in_qsize 6, out_qsize 0
2021-08-24 12:19:19,037 : 

2021-08-24 12:19:44,147 : INFO : EPOCH 51 - PROGRESS: at 53.88% examples, 87178 words/s, in_qsize 5, out_qsize 0
2021-08-24 12:19:44,909 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:19:44,921 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-08-24 12:19:44,931 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:19:44,932 : INFO : EPOCH - 51 : training on 260894 raw words (178471 effective words) took 1.8s, 97941 effective words/s
2021-08-24 12:19:46,069 : INFO : EPOCH 52 - PROGRESS: at 53.27% examples, 85194 words/s, in_qsize 5, out_qsize 0
2021-08-24 12:19:46,778 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:19:46,789 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-08-24 12:19:46,828 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:19:46,829 : INFO : EPOCH - 52 : training on 260894 raw words (178606 effective 

...

2021-08-24 12:22:36,283 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:22:36,311 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-08-24 12:22:36,335 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:22:36,336 : INFO : EPOCH - 143 : training on 260894 raw words (178778 effective words) took 1.8s, 101595 effective words/s
2021-08-24 12:22:37,473 : INFO : EPOCH 144 - PROGRESS: at 62.45% examples, 96827 words/s, in_qsize 6, out_qsize 0
2021-08-24 12:22:37,990 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:22:38,036 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-08-24 12:22:38,052 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:22:38,053 : INFO : EPOCH - 144 : training on 260894 raw words (178672 effective words) took 1.7s, 104151 effective words/s
2021-08-24 12:22:39,106 : INFO : EPOCH 145 - PROGRESS: at 58.57% e

2021-08-24 12:23:03,346 : INFO : EPOCH 159 - PROGRESS: at 62.45% examples, 100730 words/s, in_qsize 5, out_qsize 0
2021-08-24 12:23:03,842 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:23:03,870 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-08-24 12:23:03,902 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:23:03,903 : INFO : EPOCH - 159 : training on 260894 raw words (178928 effective words) took 1.7s, 108438 effective words/s
2021-08-24 12:23:04,920 : INFO : EPOCH 160 - PROGRESS: at 53.27% examples, 95374 words/s, in_qsize 5, out_qsize 0
2021-08-24 12:23:05,530 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:23:05,537 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-08-24 12:23:05,555 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:23:05,556 : INFO : EPOCH - 160 : training on 260894 raw words (178528 effe

2021-08-24 12:23:30,450 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:23:30,451 : INFO : EPOCH - 174 : training on 260894 raw words (178790 effective words) took 1.7s, 107950 effective words/s
2021-08-24 12:23:31,506 : INFO : EPOCH 175 - PROGRESS: at 58.57% examples, 98158 words/s, in_qsize 5, out_qsize 0
2021-08-24 12:23:32,049 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:23:32,086 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-08-24 12:23:32,093 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:23:32,093 : INFO : EPOCH - 175 : training on 260894 raw words (178834 effective words) took 1.6s, 109016 effective words/s
2021-08-24 12:23:33,175 : INFO : EPOCH 176 - PROGRESS: at 58.57% examples, 95694 words/s, in_qsize 6, out_qsize 0
2021-08-24 12:23:33,839 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:23:33,883 : INFO : worker thr

2021-08-24 12:23:57,920 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:23:57,957 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-08-24 12:23:57,981 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:23:57,982 : INFO : EPOCH - 190 : training on 260894 raw words (178619 effective words) took 1.7s, 106741 effective words/s
2021-08-24 12:23:59,058 : INFO : EPOCH 191 - PROGRESS: at 50.41% examples, 83821 words/s, in_qsize 5, out_qsize 0
2021-08-24 12:23:59,802 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-08-24 12:23:59,808 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-08-24 12:23:59,825 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-08-24 12:23:59,826 : INFO : EPOCH - 191 : training on 260894 raw words (178677 effective words) took 1.8s, 96980 effective words/s
2021-08-24 12:24:00,841 : INFO : EPOCH 192 - PROGRESS: at 57.14% ex

CPU times: user 17min 27s, sys: 4.95 s, total: 17min 32s
Wall time: 6min 42s


In [98]:
model.save("./doc2vec.model")

2021-08-24 12:24:41,945 : INFO : Doc2Vec lifecycle event {'fname_or_handle': './doc2vec.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-08-24T12:24:41.945643', 'gensim': '4.0.1', 'python': '3.8.10 (default, Jun  4 2021, 15:09:15) \n[GCC 7.5.0]', 'platform': 'Linux-5.13.8-100.fc33.x86_64-x86_64-with-glibc2.17', 'event': 'saving'}
2021-08-24 12:24:41,952 : INFO : not storing attribute cum_table
2021-08-24 12:24:42,004 : INFO : saved ./doc2vec.model


In [99]:
wv = model.wv
wv.save('./doc2vec.wv')

2021-08-24 12:24:44,345 : INFO : KeyedVectors lifecycle event {'fname_or_handle': './doc2vec.wv', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-08-24T12:24:44.345753', 'gensim': '4.0.1', 'python': '3.8.10 (default, Jun  4 2021, 15:09:15) \n[GCC 7.5.0]', 'platform': 'Linux-5.13.8-100.fc33.x86_64-x86_64-with-glibc2.17', 'event': 'saving'}
2021-08-24 12:24:44,366 : INFO : saved ./doc2vec.wv


In [100]:
def corpus_to_dicts(corpus):
    for doc in corpus:
        yield {
            'words': doc.words,
            'tags': doc.tags
        }

In [101]:
import simplejson


def json_save(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        simplejson.dump(data, f, separators=(',', ':'), iterable_as_array=True)

In [ ]:
# json_save(corpus_to_dicts(corpus_train), './doc2vec.corpus.train.json')

In [102]:
json_save(corpus_to_dicts(corpus_full), './doc2vec.corpus.full.json')

In [103]:
json_save(tag_id_mapping, './doc2vec.tag_id_mapping.json')

In [ ]:
# Process test corpus

In [ ]:
# df_test = pd.read_csv('./test.csv')
# df_test.shape

In [ ]:
# corpus_text_test = df_test.text.tolist()

In [ ]:
# print(corpus_text_test[-1])

In [ ]:
# def friendly_tag_corpus_test(row):
#     return row.categories.split('\n')

In [ ]:
# corpus_tags_friendly_test = df_test[["categories"]].apply(friendly_tag_corpus_test, axis=1).to_list()

In [ ]:
# corpus_tags_friendly_test[-1]

In [ ]:
# len(corpus_tags_friendly_test), len(corpus_text_test)

In [ ]:
# # weed out tags that were not seen in training
# corpus_tags_test = [[tag_id_mapping.get(tag) for tag in tags] for tags in corpus_tags_friendly_test]

In [ ]:
# corpus_tags_friendly_test[89], corpus_tags_test[89]

In [ ]:
# %%time
# corpus_words_test = corpus2tokens(corpus_text_test)

In [ ]:
# corpus_words_test[-1]

In [ ]:
# corpus_tags_test[5]

In [ ]:
# [id_tag_mapping[tag] for tag in corpus_tags_test[5] if tag]

In [ ]:
# corpus_test = gen_tagged_docs(corpus_words_test, corpus_tags_test)

In [ ]:
# json_save(corpus_to_dicts(corpus_test), './doc2vec.corpus.test.json')